In [ ]:
# This is the Sonoff Pow device running a GNL
# Listen to it (to get its IP number) with:  
# mosquitto_sub -h mqtt.local -v -t "MicroPython-3b6f59/#"
# mqtt.local=192.168.0.159


In [1]:
%serialconnect

Connecting to Serial /dev/ttyUSB3 baud=115200 
Ready.


In [1]:
# connect when going to the ESP8266 hotspot
%websocketconnect --password wpass

** WebSocket connected **
Password: 
WebREPL connected
>>> Ready.


In [9]:
# connect when going to its ip number via the router (you need this to get to mqtt.local!)
%websocketconnect --password wpass ws://192.168.0.157:8266

** WebSocket connected **
Password: Ready.


In [21]:
####################
# basic gnl code
####################
%sendtofile main.py

ssid, ssidpassword = b'DoESLiverpool', b'decafbad00'
mqttbroker = "test.mosquitto.org" # "192.168.0.159"   # mqtt.local
controlledrelay = b"ESPURNA_A7A528/relay/0/set"
clientid = "goodnightlamp" # 'MicroPython-3b6f59' # %s'%ubinascii.hexlify(network.WLAN().config('mac')[-3:]).decode()

from machine import Pin
import time, array, network
from umqtt.robust import MQTTClient

prelay = Pin(12, Pin.OUT)
pled = Pin(15, Pin.OUT)
phlwCF = Pin(14, Pin.IN)      # the power value
prelay.value(1)
power_multiplier = 10343.6

# find the power times
ptimes = array.array("I", range(0,12)) # [i, tprev, dt, dt, ..., dt]
ptimes[1] = time.ticks_ms()
ptimes[0] = 1  # avoid zero elements for recpower
def pinirq(p):
    t1 = time.ticks_ms()
    ptimes[0] += 1
    ptimes[(ptimes[0]%(len(ptimes)-2))+2] = t1 - ptimes[1]
    ptimes[1] = t1
phlwCF.irq(trigger=Pin.IRQ_RISING | Pin.IRQ_FALLING, handler=pinirq)

def recpower():
    ctime = time.ticks_ms() - ptimes[1]
    maxtime = max(ptimes[(i%(len(ptimes)-2))+2]  for i in range(ptimes[0], max(0,ptimes[0]-6),-1))
    return power_multiplier/(max(ctime, maxtime)*2)

staif = network.WLAN(network.STA_IF)
staif.active(True)
staif.connect(ssid, ssidpassword)
while not staif.isconnected():
    pled.value(1-pled.value())
    time.sleep(0.2)

c = MQTTClient(clientid, mqttbroker)

while True:
    time.sleep(3)
    pw = recpower()
    pled.value(0 if pw<5 else 1)
    print("pow", pw)
    try:
        c.connect()
    except OSError as e:
        print(e)
        continue
    c.publish((clientid+"/ip").encode(), staif.ifconfig()[0].encode())
    c.publish((clientid+"/pow").encode(), str(pw).encode())
    c.publish(controlledrelay, (b"0" if pw<5 else b"1"))
    c.disconnect()


Sent 54 lines (1713 bytes) to main.py.


In [ ]:
import os
print(os.listdir())

..........

In [16]:
# connect when going to its ip number via the router (you need this to get to mqtt.local!)
%websocketconnect --password wpass ws://192.168.0.157:8266

** WebSocket connected **
Password: 
WebREPL connected
>>> Ready.


In [4]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] 
MicroPython v1.9.3-8-g63826ac5c on 2017-11-01; ESP module with ESP8266
Type "help()" for more information.
>>> 
Closing websocket <websocket._core.WebSocket object at 0x7f0788284780>


In [5]:
%serialconnect

Connecting to Serial /dev/ttyUSB0 baud=115200 
Ready.


In [6]:
####################
# basic test interrupt times code
# listen using mosquitto_sub -h 192.168.0.159 -v -t "sonoffpow3/#"
####################

%sendtofile main.py

ssid, ssidpassword = b'DoESLiverpool', b'decafbad00'
mqttbroker = "192.168.0.159"   # mqtt.local
clientid = "sonoffpow3"

from machine import Pin
import time, array, network
from umqtt.robust import MQTTClient

prelay = Pin(12, Pin.OUT)
pled = Pin(15, Pin.OUT)
phlwCF = Pin(14, Pin.IN) # the power value
prelay.value(1)

staif = network.WLAN(network.STA_IF)
staif.active(True)
staif.connect(ssid, ssidpassword)
while not staif.isconnected():
    pled.value(1-pled.value())
    time.sleep(0.2)

c = MQTTClient(clientid, mqttbroker)

# find the power times
ptimes = array.array("I", range(0,12)) 
 # [i, tprev, dt, dt, ..., dt]
ptimes[1] = time.ticks_ms()
ptimes[0] = 1  # avoid zero elements for recpower
def pinirq(p):
    t1 = time.ticks_ms()
    i = ptimes[0]+1
    ptimes[(i%(len(ptimes)-2))+2] = t1 - ptimes[1]
    ptimes[1] = t1
    ptimes[0] = i
phlwCF.irq(trigger=Pin.IRQ_RISING | Pin.IRQ_FALLING, \
           handler=pinirq)

def mainloop():
    i0 = ptimes[0]
    tprev0 = ptimes[1]
    while True:
        time.sleep_ms(10)
        try:
            c.connect()
        except OSError as e:
            continue
        pled.value(i0%2)
        if i0 < ptimes[0]:
            c.publish(b"sonoffpow2/tick", b"%d %d"%\
                      (i0, ptimes[(i0%(len(ptimes)-2))+2]))
            i0 += 1
        if ptimes[1] - tprev0 > 5000:
            c.publish(b"sonoffpow2/ip", \
                staif.ifconfig()[0].encode())
            print(i0, tprev0)
            tprev0 = ptimes[1]
            
mainloop()


Sent 57 lines (1518 bytes) to main.py.


In [3]:
%rebootdevice

normal repl mode not detected b'\r\nMicroPython v1.9.3-8-g63826ac5c on 2017-11-01; ESP module with ESP8266\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nPYB: sof#5 ets_task(40100130, 3, 3fff837c, 4)\r\nWebREPL is not configured, run \'import webrepl_setup\'\r\n#6 ets_task(4020ed88, 28, 3fff97e8, 10)\r\n'
not command mode
[Timed out waiting for recognizable response]


In [7]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.9.3-8-g63826ac5c on 2017-11-01; ESP module with ESP8266\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7f16a11eb1d0, open=True>(port='/dev/ttyUSB0', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [18]:
%rebootdevice

normal repl mode not detected 
not command mode
[Timed out waiting for recognizable response]


In [ ]:
# the remaining code is scratchspace for working out the main.py file above

In [23]:
import os
print(os.statvfs(""))
print(os.listdir(""))
for f in os.listdir(""):
     print("File: {} stats: {}".format(f, os.stat(f)))


(4096, 4096, 860, 0, 0, 0, 0, 0, 0, 255)
['boot.py', 'webrepl_cfg.py', 'test.py']
File: boot.py stats: (32768, 0, 0, 0, 0, 0, 160, 54, 54, 54)
File: webrepl_cfg.py stats: (32768, 0, 0, 0, 0, 0, 15, 54, 54, 54)
File: test.py stats: (32768, 0, 0, 0, 0, 0, 0, 1940, 1940, 1940)


In [3]:
from machine import Pin

prelay = Pin(12, Pin.OUT)
pled = Pin(15, Pin.OUT)
pbutton = Pin(0, Pin.IN)

phlwselCF1 = Pin(5, Pin.OUT)  # 0 current, 1 voltage
phlwCF1 = Pin(13, Pin.IN)
phlwCF = Pin(14, Pin.IN)      # the power value


In [5]:
pled.value(0)

In [6]:
prelay.value(1)

In [24]:
phlwselCF1.value(0)

In [68]:
import time, array
ptimes = array.array("I", range(0,12)) # [i, tprev, dt, dt, ..., dt]
ptimes[1] = time.ticks_ms()
ptimes[0] = 1  # avoid zero elements for recpower
def pinirq(p):
    t1 = time.ticks_ms()
    ptimes[0] += 1
    ptimes[(ptimes[0]%(len(ptimes)-2))+2] = t1 - ptimes[1]
    ptimes[1] = t1

phlwCF.irq(trigger=Pin.IRQ_RISING | Pin.IRQ_FALLING, handler=pinirq)


In [18]:
import ubinascii; O6 = ubinascii.a2b_base64

In [74]:
prelay.value(1)

In [7]:

# calculate the vital statistics
V_REF = 2.43
F_OSC = 3579000
HLW8012_CURRENT_R = 0.001 # 1milliOhm copper-manganese resistor in series with the main line
HLW8012_VOLTAGE_R_DOWN = 1000 # 1kOhm resistor in the voltage divider that feeds the V2P
HLW8012_VOLTAGE_R_UP = 5 * 470000 # 5 470kOhm resistors in the voltage divider that feeds the V2P pin
current_resistor = HLW8012_CURRENT_R
voltage_resistor = (HLW8012_VOLTAGE_R_UP + HLW8012_VOLTAGE_R_DOWN) / HLW8012_VOLTAGE_R_DOWN
current_multiplier = (1000.0 * 512 * V_REF / current_resistor / 24.0 / F_OSC)
voltage_multiplier = (1000.0 * 512 * V_REF * voltage_resistor / 2.0 / F_OSC)
power_multiplier = (1000.0 * 128 * V_REF * V_REF * voltage_resistor / current_resistor / 48.0 / F_OSC)
print(power_multiplier)
    

...

*** Sending Ctrl-C

[Timed out waiting for recognizable response]


In [112]:
def ytimes():
    i = ptimes[0]
    while True:
        while i < ptimes[0]:
            i += 1
            yield ptimes[(i%(len(ptimes)-2))+2]
        time.sleep_ms(10)

# I think the rolling max value is the best as avoids the noise on lowe power 
# and goes out of scope on high power (lots of low values)
def recpower():
    ctime = time.ticks_ms() - ptimes[1]
    maxtime = max(ptimes[(i%(len(ptimes)-2))+2]  for i in range(ptimes[0], max(0,ptimes[0]-6),-1))
    return max(ctime, maxtime)
    

In [21]:
import os
f = open("test.py", "w")
f.write("hi there")

Traceback (most recent call last):
  File "<stdin>", line 3, in <module>
OSError: 28


In [25]:
%sendtofile test.py
print(power_multiplier/(recpower()*2))

Sent 1 lines (38 bytes) to test.py.


Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
OSError: 28


In [20]:
help("modules")

__main__          http_client_ssl   sys               urandom
_boot             http_server       time              ure
_onewire          http_server_ssl   uasyncio/__init__ urequests
_webrepl          inisetup          uasyncio/core     urllib/urequest
apa102            json              ubinascii         uselect
array             lwip              ucollections      usocket
btree             machine           uctypes           ussl
builtins          math              uerrno            ustruct
dht               micropython       uhashlib          utime
ds18x20           neopixel          uheapq            utimeq
errno             network           uio               uzlib
esp               ntptime           ujson             webrepl
example_pub_button                  onewire           umqtt/robust      webrepl_setup
example_sub_led   os                umqtt/simple      websocket
flashbdev         port_diag         uos               websocket_helper
framebuf          select            u

In [146]:
import machine
print(machine.unique_id())

b'Yo;\x00'


In [25]:
%websocketconnect --password wpass ws://192.168.4.1:8266

** WebSocket connected **
Password: 
WebREPL connected
>>> Ready.


In [26]:
import os
print(os.listdir())


['boot.py', 'webrepl_cfg.py']


In [27]:
from machine import Pin

prelay = Pin(12, Pin.OUT)
pled = Pin(15, Pin.OUT)
pbutton = Pin(0, Pin.IN)


In [29]:
prelay.value(0)

In [18]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] 
MicroPython v1.9.3-8-g63826ac5c on 2017-11-01; ESP module with ESP8266
Type "help()" for more information.
>>> 
Closing websocket <websocket._core.WebSocket object at 0x7f6440591ef0>


In [ ]:
c.publish(b"ESPURNA_A7A528/relay/0/set", b"1")
